In [ ]:
import xmlrpc.client
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import platform

load_dotenv()

dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]

common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(dbUrl))
uid = common.authenticate(dbName, dbUsername, dbPassword, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(dbUrl))

# Data Handling Functions
def id_field_into_1value(jsonFile):
    for index, line in enumerate(jsonFile):
        for item in line:
            if type(line[item]) == list:
                jsonFile[index][item] = jsonFile[index][item][1]

df = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order',
    'search_read',
    [],
    {'fields': ["partner_id", "name", "stage_id", "pricelist_id", "recurrence_id", "user_id", "company_id", "x_partner_en_sale_order", "create_date"] })

id_field_into_1value(df)

df = pd.DataFrame(df)

# Create a dictionary for currency conversion
currency_mapping = {
    #"MXN": "MXN",
    "USD": "USD",
    "BRL": "BRL",
    "ARS": "ARS",
    "CLP": "CLP",
    "COP": "COP",
    "PEN": "PEN"
}

usd_er = {}
for curr in currency_mapping.keys():
    curr_er = yf.Ticker(curr + "=X")
    curr_er.info["regularMarketPreviousClose"]
    usd_er[curr] = curr_er.info["regularMarketPreviousClose"]

# Extract the currency from the 'pricelist_id' column
df['pricelist_id'] = df['pricelist_id'].str.extract(r'\((.*?)\)', expand=False)

# Map the full currency names to their three-letter ISO codes
df['pricelist_id'] = df['pricelist_id'].map(currency_mapping)

# Map the exchange rates to a new column based on 'pricelist_id'
df['exchange_rate'] = df['pricelist_id'].map(usd_er)

df = df.rename(columns={'id': 'order_id', 'name': 'order_name', "partner_id": "customer_id"})

df3 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order.stage',
    'search_read',
    [],
    {'fields': ["name", "category"] })

id_field_into_1value(df3)

df3 = pd.DataFrame(df3)

df3 = df3.drop(columns=['id'])

df = df.merge(df3, how='left', left_on='stage_id', right_on='name')

df2 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order.line',
    'search_read',
    [],
    {'fields': ["order_id", "product_id", "product_uom_qty", "qty_delivered", "price_unit", "price_subtotal","recurring_monthly", "discount"] })

id_field_into_1value(df2)

df2 = pd.DataFrame(df2)

df2 = df2.rename(columns={'id': 'line_id', "order_id": "order_name"})

df4 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'product.template',
    'search_read',
    [],
    {'fields': ["name", "recurring_invoice"]})

id_field_into_1value(df4)

df4 = pd.DataFrame(df4)

df4 = df4.drop(columns=['id'])

df2['temp_join'] = df2['product_id'].str.replace(r' \(.*?\)', '', regex=True).str.strip()

df2 = df2.merge(df4, how='left', left_on='temp_join', right_on='name')

df2 = df2.drop(columns=['temp_join'])

df2 = df2.drop(columns=['name'])

# Left join df and df2 on the column order_id and order_name
export_df = df.merge(df2, how='left', left_on='order_name', right_on='order_name')

# Divide 'recurring_monthly' by 'exchange_rate' to convert to USD
export_df['recurring_monthly'] = round(export_df['recurring_monthly'].astype(float) / export_df['exchange_rate'], 2)

DATABASE_CONNECTION_URI = os.environ["DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

# insert the dataframe into the postgres table
export_df.to_sql('odoo_subscriptions', con=engine, if_exists='replace', index=False)

